### Day 16

In [265]:
import numpy as np
from math import prod
from collections import defaultdict, Counter

In [2]:
day_i = 16

In [3]:
%run start_day.py $day_i

Initializing day 16


In [4]:
cd /home/vincent/Documents/AdventOfCode/2021

/home/vincent/Documents/AdventOfCode/2021


In [5]:
PATH = f"day{day_i}/input{day_i}"

In [6]:
!wc -l $PATH

1 day16/input16


In [7]:
!head $PATH

020D790050D26C13EC1348326D336ACE00EC299E6A8B929ED59C880502E00A526B969F62BF35CB4FB15B93A6311F67F813300470037500043E2D4218FA000864538E905A39CAF77386E35AB01802FC01BA00021118617C1F00043A3F4748A53CF66008D00481272D73308334EDB0ED304E200D4E94CF612A49B40036C98A7CF24A53CA94C6370FBDCC9018029600ACD529CA9A4F62ACD2B5F928802F0D2665CA7D6CC013919E78A3800D3CF7794A8FC938280473057394AFF15099BA23CDD37A08400E2A5F7297F916C9F97F82D2DFA734BC600D4E3BC89CCBABCBE2B77D200412599244D4C0138C780120CC67E9D351C5AB4E1D4C981802980080CDB84E034C5767C60124F3BC984CD1E479201232C016100662D45089A00087C1084F12A724752BEFEA9C51500566759BF9BE6C5080217910CD00525B6350E8C00E9272200DCE4EF4C1DD003952F7059BCF675443005680103976997699795E830C02E4CBCE72EFC6A6218C88C9DF2F3351FCEF2D83CADB779F59A052801F2BAACDAE7F52A8190073937FE1D700439234DBB4F7374DC0CC804CF1006A0D47B8A4200F445865170401F8251662D100909401AB8803313217C680004320D43F871308D140C010E0069E7EDD1796AFC8255800052E20043E0F42A8B6400864258E51088010B85910A0F4ECE1DFE069C0229AE63D0B8DC6F8252940

In [ ]:
!tail $PATH

In [ ]:
str

In [19]:
str(bin(int("0", 16)))[2:].ljust(4, '0')

'0000'

In [24]:
def parse_input(inp):
    s = ""
    for c in inp[0]:
        s += str(bin(int(c, 16)))[2:].rjust(4, '0')
    return s

In [25]:
# real input
with open(PATH, 'r') as f:
    inputs = parse_input([x.strip() for x in f.readlines()])


In [130]:
# test input
test_str1 = ["38006F45291200"]
test_str2 = ["EE00D40C823060"]

test_str3 = ["8A004A801A8002F478"]
test_str4 = ["620080001611562C8802118E34"]
test_str5 = ["C0015000016115A2E0802F182340"]
test_str6 = ["A0016C880162017C3686B18A3D4780"]


inputs_1 = parse_input(test_str1)
inputs_2 = parse_input(test_str2)
inputs_3 = parse_input(test_str3)
inputs_4 = parse_input(test_str4)
inputs_5 = parse_input(test_str5)
inputs_6 = parse_input(test_str6)

In [71]:
inputs_1

'00111000000000000110111101000101001010010001001000000000'

In [268]:
def read_literal(p):
    i = 6
    group = p[i:i+5]
    binary = ""
    while group[0] == '1':
        binary += group[1:]
        i += 5
        group = p[i:i+5]
    binary += group[1:]
#     end = (i + 4 + 3 - (i % 4))
    end = i + 5
    return int(binary, 2), end


class Packet():
    OPERATIONS = [sum, prod, min, max]
    
    def __init__(self, p, debug=False):
        if len(p) > 6: self.version, self.ID = int(p[:3], 2), int(p[3:6], 2)
        else: self.version, self.ID = None, None
        self.value = None
        self.subpackets = []
        self.debug = debug
        
        if self.debug: 
            print(f"PARSING {p if len(p) < 40 else (p[:38] + '...')}")
            if len(p) > 6:
                print(f"        VVVTTT{'AAAAA' if self.ID == 4 else ('I' + 'L' * (15 if p[6] == '0' else 11))}")
            print(f"\tversion {self.version}\n\tID {self.ID}")
        self.parse_content(p, self.ID)
    
    def parse_content(self, content, ID):
        if ID == 4:
            self.value, self.end_position = read_literal(content)
            if self.debug: print(f'\tvalue - {self.value}\n\tend - {self.end_position}')
        elif content == '' or int(content, 2) == 0 or self.version is None:
            self.end_position = len(content) - 1
        else:
            length_type_ID = content[6]
            assert len(content) > 6, f"Cannot find length_type_ID in {content}"
            if self.debug: print(f"\tlength type {length_type_ID}")
            if length_type_ID == '1':
                assert len(content) > 18, f"Content too short for length_type_ID 1 - {content}"
                number_of_subpackets = int(content[7:18], 2)
                if self.debug: print(f"\tnumber of subpackets {number_of_subpackets}")
                start = 18
                for i in range(number_of_subpackets):
                    subpacket = Packet(content[start:], self.debug)
                    if subpacket.end_position == 0:
                        break
                    self.subpackets.append(subpacket)
                    start += subpacket.end_position
                self.end_position = start
            else:
                assert len(content) > 22, f"Content too short for length_type_ID 1 - {content}"
                total_length_in_bits = int(content[7:22], 2)
                if self.debug: print(f"\tnumber of bits {total_length_in_bits}")
                self.end_position = 22 + total_length_in_bits
                start = 22
                while start < self.end_position:
                    subpacket = Packet(content[start:self.end_position], self.debug)
                    if subpacket.end_position == 0:
                        break
                    self.subpackets.append(subpacket)
                    start += subpacket.end_position
    
    def sum_of_versions(self):
        return self.version + sum([p.sum_of_versions() for p in self.subpackets])
    
    def to_string(self, offset = ""):
        if self.value is None:
            return offset + f'ID {self.ID}:\n' + '\n'.join([sub.to_string(offset + '\t') for sub in self.subpackets])
        else:
            return offset + f'value = {self.value}'
        
    def evaluate(self):
        if self.value is not None:
            return self.value
        elif self.ID < 4:
            return self.OPERATIONS[self.ID](x.evaluate() for x in self.subpackets)
        elif self.ID == 5:
            values = [x.evaluate() for x in self.subpackets]
            return int(values[0] > values[1])
        elif self.ID == 6:
            values = [x.evaluate() for x in self.subpackets]
            return int(values[0] < values[1])
        elif self.ID == 7:
            values = [x.evaluate() for x in self.subpackets]
            return int(values[0] == values[1])
        


def solve1(inp, debug=False):
    p = Packet(inp, debug)
    return p.sum_of_versions()

def solve2(inp, debug=False):
    p = Packet(inp, debug)
    return p.evaluate()

In [169]:
inputs_1

'00111000000000000110111101000101001010010001001000000000'

In [170]:
inputs_1[22:]

'1101000101001010010001001000000000'

In [252]:
Packet(inputs_1, debug=True)

PARSING 00111000000000000110111101000101001010...
        VVVTTTILLLLLLLLLLLLLLL
	version 1
	ID 6
	length type 0
	number of bits 27
PARSING 110100010100101001000100100
        VVVTTTAAAAA
	version 6
	ID 4
	value - 10
	end - 11
PARSING 0101001000100100
        VVVTTTAAAAA
	version 2
	ID 4
	value - 20
	end - 16


In [253]:
print(Packet(inputs_1).to_string())

ID 6:
	value = 10
	value = 20


In [254]:
Packet(inputs_2, debug=True)

PARSING 11101110000000001101010000001100100000...
        VVVTTTILLLLLLLLLLL
	version 7
	ID 3
	length type 1
	number of subpackets 3
PARSING 01010000001100100000100011000001100000
        VVVTTTAAAAA
	version 2
	ID 4
	value - 1
	end - 11
PARSING 100100000100011000001100000
        VVVTTTAAAAA
	version 4
	ID 4
	value - 2
	end - 11
PARSING 0011000001100000
        VVVTTTAAAAA
	version 1
	ID 4
	value - 3
	end - 11


In [255]:
print(Packet(inputs_2).to_string())

ID 3:
	value = 1
	value = 2
	value = 3


In [256]:
print(solve1(inputs_3))

16


In [257]:
print(solve1(inputs_4))

12


In [258]:
print(solve1(inputs_5, debug=True))

PARSING 11000000000000010101000000000000000000...
        VVVTTTILLLLLLLLLLLLLLL
	version 6
	ID 0
	length type 0
	number of bits 84
PARSING 00000000000000000101100001000101011010...
        VVVTTTILLLLLLLLLLLLLLL
	version 0
	ID 0
	length type 0
	number of bits 22
PARSING 0001000101011010001011
        VVVTTTAAAAA
	version 0
	ID 4
	value - 10
	end - 11
PARSING 11010001011
        VVVTTTAAAAA
	version 6
	ID 4
	value - 11
	end - 11
PARSING 10000010000000001011110001100000100011...
        VVVTTTILLLLLLLLLLL
	version 4
	ID 0
	length type 1
	number of subpackets 2
PARSING 1111000110000010001101
        VVVTTTAAAAA
	version 7
	ID 4
	value - 12
	end - 11
PARSING 00010001101
        VVVTTTAAAAA
	version 0
	ID 4
	value - 13
	end - 11
23


In [259]:
print(Packet(inputs_5).to_string())

ID 0:
	ID 0:
		value = 10
		value = 11
	ID 0:
		value = 12
		value = 13


In [260]:
print(Packet(inputs_6).to_string())

ID 0:
	ID 0:
		ID 0:
			value = 6
			value = 6
			value = 12
			value = 15
			value = 15


In [261]:
print(solve1(inputs_6))

31


In [248]:
int('00111100', 2)

60

In [262]:
Packet(inputs, True)

PARSING 00000010000011010111100100000000010100...
        VVVTTTILLLLLLLLLLL
	version 0
	ID 0
	length type 1
	number of subpackets 53
PARSING 11100100000000010100001101001001101100...
        VVVTTTILLLLLLLLLLLLLLL
	version 7
	ID 1
	length type 0
	number of bits 80
PARSING 11010010011011000001001111101100000100...
        VVVTTTAAAAA
	version 6
	ID 4
	value - 60
	end - 16
PARSING 00010011111011000001001101001000001100...
        VVVTTTAAAAA
	version 0
	ID 4
	value - 252
	end - 16
PARSING 00010011010010000011001001101101001100...
        VVVTTTAAAAA
	version 0
	ID 4
	value - 168
	end - 16
PARSING 00110010011011010011001101101010
        VVVTTTAAAAA
	version 1
	ID 4
	value - 61
	end - 16
PARSING 0011001101101010
        VVVTTTAAAAA
	version 1
	ID 4
	value - 186
	end - 16
PARSING 11001110000000001110110000101001100111...
        VVVTTTILLLLLLLLLLL
	version 6
	ID 3
	length type 1
	number of subpackets 3
PARSING 10110000101001100111100110101010001011...
        VVVTTTAAAAA
	version 5
	ID 4


In [263]:
print(Packet(inputs).to_string())

ID 0:
	ID 1:
		value = 60
		value = 252
		value = 168
		value = 61
		value = 186
	ID 3:
		value = 5
		value = 237
		value = 11
	value = 322158
	ID 1:
		ID 7:
			value = 180763931
			value = 180763931
		value = 225622007682
	ID 1:
		ID 6:
			ID 0:
				value = 11
				value = 4
				value = 15
			ID 0:
				value = 10
				value = 14
				value = 2
		value = 14
	value = 402435806556
	ID 1:
		ID 7:
			ID 0:
				value = 12
				value = 5
				value = 3
			ID 0:
				value = 15
				value = 14
				value = 10
		value = 239
	ID 1:
		ID 5:
			value = 850169868
			value = 54926
		value = 132
	ID 0:
		value = 213
		value = 3721
		value = 45
	ID 0:
		value = 34039
		value = 140186
		value = 8958207
		value = 148
	ID 1:
		ID 5:
			value = 10767446
			value = 59484
		value = 22004
	ID 0:
		value = 6
		value = 9618683
	ID 1:
		value = 80330
		ID 6:
			value = 15582291
			value = 46
	ID 2:
		value = 133
		value = 205912145
		value = 2888
		value = 740257
	ID 2:
		value = 2065890333
		value = 995266
		value = 

In [264]:
solve1(inputs)

906

### Test part 2

In [269]:
tests = [
    "C200B40A82",
    "04005AC33890",
    "880086C3E88112",
    "CE00C43D881120",
    "D8005AC2A8F0",
    "F600BC2D8F",
    "9C005AC2F8F0",
    "9C0141080250320F1802104A08"
]

for t in tests:
    print(solve2(parse_input([t])))

3
54
7
9
1
0
0
1


In [270]:
solve2(inputs)

819324480368

In [ ]:
solve2()